**Modèle d'entraînement à executer une fois au début**

In [ ]:
# --- CELLULE 1 : SETUP / ENTRAINEMENT (run once) ---
# Exécute uniquement la première fois ou si tu veux réentraîner le modèle.

import pandas as pd
import gradio as gr
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
# --- 1️⃣ Charger le dataset ---
sheet_url = "https://docs.google.com/spreadsheets/d/1PIMO76csrwWa7eRCZHQziJQvqGMy7M_EpZLblZjRFMU/export?format=csv"
df = pd.read_csv(sheet_url)

In [ ]:
# --- Colonnes ---
categorical_cols = ['deplacements_pro', 'domaine_detude', 'poste']
binary_cols = ['genre', 'situation', 'heures_sup']
numeric_cols = [
    'age', 'trajet_quotidien_en_miles', 'salaire_mensuel', 'num_nb_entreprises',
    'num_pourcentage_augmentation', 'num_performance', 'num_equilibre_pro_perso',
    'num_temps_de_formation', 'annees_dans_entreprise', 'annees_poste_actuel',
    'annees_depuis_promotion', 'num_annees_avec_manager',
    'num_satisfaction_environnement', 'num_satisfaction_travail', 'num_implication',
    'num_satisfaction_relation', 'tt_annee_travail', 'num_taux_daction',
    'num_niveau_detude', 'statut', 'num_niveau_hierarchique'
]

# --- Encodage colonnes binaires ---
df['genre'] = df['genre'].map({'Femme': 0, 'Homme': 1}).fillna(0).astype(int)
df['heures_sup'] = df['heures_sup'].map({'Non': 0, 'Oui': 1}).fillna(0).astype(int)
df['situation'] = df['situation'].map({
    'Célibataire': 0, 'Divorcé(e)': 0, 'Divorcé': 0, 'Marié(e)': 1, 'Marié': 1
}).fillna(0).astype(int)

# --- Target / features ---
X = df[categorical_cols + binary_cols + numeric_cols]
y = df['num_attrition']

# --- Split train/test ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# --- Préprocessing ---
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'
)

# --- Prétraitement + SMOTE ---
X_train_preprocessed = preprocessor.fit_transform(X_train)
sm = SMOTE(sampling_strategy=0.5, random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train_preprocessed, y_train)

# --- Entraînement modèle ---
model = LogisticRegression(max_iter=1000)
model.fit(X_train_res, y_train_res)

# --- Pipeline final ---
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# --- Sauvegarde ---
joblib.dump(pipeline, 'pipeline_attrisk.joblib')
df.to_pickle('df_for_ui.pkl')

# --- Mappings pour UI ---
niveau_hierarchique_mapping = {'Débutant':0, 'Confirmé':1, 'Senior':2, 'Manager':3, 'Directeur':4}
niveau_etude_mapping = {'Baccalauréat':0, 'Bac+2':1, 'Licence':2, 'Master':3, 'Doctorat':4}
genre_mapping = {'Femme':0, 'Homme':1}
heures_sup_mapping = {'Non':0, 'Oui':1}
situation_mapping = {'Célibataire':0, 'Divorcé':0, 'Marié':1}

joblib.dump({
    'niveau_hierarchique_mapping': niveau_hierarchique_mapping,
    'niveau_etude_mapping': niveau_etude_mapping,
    'genre_mapping': genre_mapping,
    'heures_sup_mapping': heures_sup_mapping,
    'situation_mapping': situation_mapping
}, 'mappings_attrisk.joblib')

print("✅ Entraînement terminé et pipeline sauvegardé")

✅ Entraînement terminé et pipeline sauvegardé


# **Modèle Gradio**

In [9]:
# --- Couleurs & tailles ---
PRIMARY_COLOR = "#0047AB"
TEXT_ON_PRIMARY = "#FFFFFF"
HOVER_COLOR = "#003380"
GLOBAL_FONT_SIZE_PX = 18
OUTPUT_FONT_SIZE_PX = 20
THERMOMETER_HEIGHT_PX = 25

# --- CSS personnalisé ---
custom_css = f"""
.gradio-container * {{
    font-size: {GLOBAL_FONT_SIZE_PX}px !important;
}}
h1 {{
    font-size: 45px !important;
    font-weight: 700;
}}
.gr-accordion summary {{
    font-size: 24px !important;
    font-weight: 600;
}}
button, .gr-button {{
    background-color: {PRIMARY_COLOR} !important;
    color: {TEXT_ON_PRIMARY} !important;
    border-radius: 8px !important;
    padding: 10px 16px !important;
    font-weight: 600 !important;
}}
button:hover, .gr-button:hover {{
    background-color: {HOVER_COLOR} !important;
    transform: translateY(-1px);
    transition: all 0.12s ease;
}}
.gradio-container .output_textbox textarea {{
    font-size: {OUTPUT_FONT_SIZE_PX}px !important;
}}
"""

def prepare_inputs(
    age, deplacements, trajet, niveau_etude_text, domaine, sat_env, genre_text,
    implication, poste, sat_travail, situation_text, salaire,
    nb_entreprises, heures_sup_text, augmentation, performance, sat_relation,
    taux_action, equilibre, temps_formation, anc_entreprise, anc_poste,
    promo, manager, statut, niveau_hierarchique_text, annee_carriere
):
    niveau_hier_num = niveau_hierarchique_mapping.get(niveau_hierarchique_text, 0)
    niveau_etude_num = niveau_etude_mapping.get(niveau_etude_text, 0)
    genre_num = genre_mapping.get(genre_text, 1)
    heures_sup_num = heures_sup_mapping.get(heures_sup_text, 0)
    situation_num = situation_mapping.get(situation_text, 0)

    input_dict = {
        'age': age,
        'deplacements_pro': deplacements,
        'trajet_quotidien_en_miles': trajet,
        'num_niveau_detude': niveau_etude_num,
        'domaine_detude': domaine,
        'genre': genre_num,
        'situation': situation_num,
        'poste': poste,
        'salaire_mensuel': salaire,
        'num_nb_entreprises': nb_entreprises,
        'heures_sup': heures_sup_num,
        'num_pourcentage_augmentation': augmentation,
        'num_performance': performance,
        'num_equilibre_pro_perso': equilibre,
        'num_temps_de_formation': temps_formation,
        'annees_dans_entreprise': anc_entreprise,
        'annees_poste_actuel': anc_poste,
        'annees_depuis_promotion': promo,
        'num_annees_avec_manager': manager,
        'statut': statut,
        'num_satisfaction_environnement': sat_env,
        'num_satisfaction_travail': sat_travail,
        'num_implication': implication,
        'num_satisfaction_relation': sat_relation,
        'num_taux_daction': taux_action,
        'num_niveau_hierarchique': niveau_hier_num,
        'tt_annee_travail': annee_carriere   # <-- ajoutée
    }

    input_df = pd.DataFrame([input_dict])
    proba = pipeline.predict_proba(input_df)[0][1]
    pred = pipeline.predict(input_df)[0]

    # Thermomètre de risque
    color = "#4CAF50" if proba<0.5 else "#FFC107" if proba<0.6 else "#F44336"
    html_thermo = f"""
    <div style='width:100%; background:#EEE; border-radius:5px;'>
        <div style='width:{proba*100:.1f}%; background:{color}; height:{THERMOMETER_HEIGHT_PX}px; border-radius:5px; text-align:center; color:white; font-weight:bold;'>
            {proba:.0%}
        </div>
    </div>
    """
    verdict = "⚠️ RISQUE ÉLEVÉ de départ" if pred==1 else "✅ Faible risque de départ"
    return f"<b>{verdict}</b><br><br>{html_thermo}"


    # --- Interface Gradio avec accordéons et dropdowns groupés ---
with gr.Blocks(title="🔍AttriScan©", css=custom_css) as demo:
    gr.HTML("""
    <div style='text-align:center; margin-bottom:12px;'>
        <h1>🔍AttriScan<sup>©</sup></h1>
        <p style='font-size:18px; margin-top:6px;'>Tous les champs sont préremplis avec des valeurs par défaut.</p>
    </div>
    """)

    # Accordeon : Personal
    with gr.Accordion("👤 Informations personnelles", open=True):
        with gr.Row():
            age = gr.Number(label="Âge", value=int(df['age'].median()))
            genre = gr.Radio(label="Genre", choices=list(genre_mapping.keys()), value="Homme")
            situation = gr.Radio(label="Situation", choices=list(situation_mapping.keys()), value="Marié")
        with gr.Row():
            trajet = gr.Number(label="Trajet quotidien en miles", value=int(df['trajet_quotidien_en_miles'].median()))
            nb_entreprises = gr.Number(label="Nombre d'entreprises précédentes", value=int(df['num_nb_entreprises'].median()))
            deplacements = gr.Radio(label="Déplacements", choices=df['deplacements_pro'].unique().tolist(), value=df['deplacements_pro'].mode()[0])

    # Accordeon : Salaire & Avantages
    with gr.Accordion("💼 Salaire & avantages", open=False):
        with gr.Row():
            salaire = gr.Number(label="Salaire mensuel", value=int(df['salaire_mensuel'].median()))
            augmentation = gr.Number(label="Pourcentage augmentation", value=int(df['num_pourcentage_augmentation'].median()))
        with gr.Row():
            heures_sup = gr.Radio(label="Heures sup", choices=list(heures_sup_mapping.keys()), value="Non")
            temps_formation = gr.Number(label="Temps de formation", value=int(df['num_temps_de_formation'].median()))
            taux_action = gr.Radio(label="Taux d'action", choices=[0,1,2,3], value=int(df['num_taux_daction'].median()))

    # Accordeon : Education & Domaine
    with gr.Accordion("🎓 Études & domaine", open=False):
        niveau_etude = gr.Dropdown(label="Niveau d'étude", choices=list(niveau_etude_mapping.keys()), value='Bac+2')
        domaine = gr.Dropdown(label="Domaine d'étude", choices=df['domaine_detude'].unique().tolist(), value=df['domaine_detude'].mode()[0])

    # Accordeon : Carrière & Performance
    with gr.Accordion("📈 Performance & carrière", open=False):
        with gr.Row():
            poste = gr.Dropdown(label="Poste", choices=df['poste'].unique().tolist(), value=df['poste'].mode()[0])
            niveau_hierarchique = gr.Dropdown(label="Niveau hiérarchique", choices=list(niveau_hierarchique_mapping.keys()), value='Confirmé')
            statut = gr.Radio(label="Statut", choices=[1,2,3], value=int(df['statut'].median()))
        with gr.Row():
            anc_entreprise = gr.Number(label="Années dans l'entreprise", value=int(df['annees_dans_entreprise'].median()))
            annee_carriere = gr.Number(label="Années de carrière", value=int(df['tt_annee_travail'].median()))
            anc_poste = gr.Number(label="Années sur le poste actuel", value=int(df['annees_poste_actuel'].median()))
        with gr.Row():
            promo = gr.Number(label="Années depuis dernière promotion", value=int(df['annees_depuis_promotion'].median()))
            manager = gr.Number(label="Années avec manager", value=int(df['num_annees_avec_manager'].median()))

    # Accordeon : Satisfaction & Engagement
    with gr.Accordion("🙂 Satisfaction & engagement", open=False):
        sat_env = gr.Radio(label="Satisfaction environnement", choices=[1,2,3,4], value=int(df['num_satisfaction_environnement'].median()))
        sat_travail = gr.Radio(label="Satisfaction travail", choices=[1,2,3,4], value=int(df['num_satisfaction_travail'].median()))
        equilibre = gr.Radio(label="Équilibre pro/perso", choices=[1,2,3,4], value=int(df['num_equilibre_pro_perso'].median()))
        implication = gr.Radio(label="Implication", choices=[1,2,3,4], value=int(df['num_implication'].median()))
        performance = gr.Radio(label="Performance", choices=[1,2,3,4], value=int(df['num_performance'].median()))
        sat_relation = gr.Radio(label="Satisfaction relation", choices=[1,2,3,4], value=int(df['num_satisfaction_relation'].median()))

    # Résultat et bouton
    output_box = gr.HTML()
    bouton = gr.Button("🔮 Calculer le risque d'attrition")
    bouton.click(
        fn=prepare_inputs,
        inputs=[age, deplacements, trajet, niveau_etude, domaine, sat_env, genre,
                implication, poste, sat_travail, situation, salaire,
                nb_entreprises, heures_sup, augmentation, performance, sat_relation,
                taux_action, equilibre, temps_formation, anc_entreprise, anc_poste,
                promo, manager, statut, niveau_hierarchique, annee_carriere],
        outputs=output_box
    )

# Lancement
demo.launch()

/tmp/ipython-input-552399826.py:100: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title="🔍AttriScan©", css=custom_css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce2f0729d4fa6f55a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
